## Basics of Molecular Dynamics

The goal is to generate enough representative conformations of the molecular system in such a way that accurate values of a property can be obtained.

Molecular Dynamics (MD) generates configurations by applying Newton's equation of motion $\frac{d^2r_i}{dt^2}=\frac{F_i}{m_i}$. So, we start with a conformation ($r_i,t_i$) and then use Newton's equation of motion to move atoms ina molecular system to go to a new conformation ($r_j,t_j$) and so on.

<img src="./conf1.png" width="400" height="200">

$m_i$ = mass of the $i^{th}$ atom which we know _apriori_ and the force can be calculated as $F_i = -\frac{\partial}{\partial r_i}V(r_i,r_2,\ldots,r_N)$ = - slope of the potential. The potential defined analytically using classical mechanics and contains interactions that depend on bonds, angles, torsions, etc. A force field defined in this way is called a **molecular mechanics force field**. Each force field has its own analytical functions and a set of predefined parameters. A typical force field has the following form

$V(r_1,r_2,\ldots,r_N) = \sum_{bonds}\frac{1}{2}K_b(b-b_0)^2 + \sum_{angles}\frac{1}{2}K_{\theta}(\theta - \theta_0)^2 + \sum_{torsions} \frac{1}{2}K_\xi (\xi - \xi_0)^2 + \frac{1}{2}K_{\phi}[1+\cos(n\phi+\delta)]\quad \text{bonded interactions} \\
\quad\quad\quad\quad\quad\quad+\sum_{pairs}[\frac{C_{12}(i,j)}{r_{ij}^{12}}-\frac{C_{6}(i,j)}{r_{ij}^{6}}] \text{ (Lennard-Jones) }+ \frac{q_iq_j}{4\pi\epsilon_0\epsilon r_{ij}} \text{ (Coulombic) }\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\text{non-bonded interactions}$

Most common force field families: AMBER, CHARMM, GROMOS, OPLS, Martini, etc.


## Input files for MD Simulations

* Informations on the **positions (and velocities)** of the atoms :: **structure file (\*pdb,\*gro)**
 * initial coordinate (from expt. or build the guess structure) and the solvent box.
  * Possible problems: not all atom positions are know, hydrogen atom positions, trapped water or ions, extra molecules needed for experimental analysis, etc.
  * Possible fixes: look at the pdb structure and 1. add missing atoms (if any), 2. set protonation/termini using built in tools from gromacs (_gmx_ and _pdb2gmx_). Then put the system in a box (as periodic boundary condition is used) using _gmx editconf_ (this command defines a simulation box). <img src="./periodic-boundary-cond2.png" width="600" height="200"> then we will also have to select the box shape, e.g., cubic, rectangular, hexagonal, truncated otahedron, rhombic dodecahedron, etc. Then we need to include the environment using _gmx solvate_ and _gmx genion_.
  
* Information about the **molecular model** :: **topology file (\*top, \*itp)**
 * Now we need to generate the topology file. First, we choose a force field: we need the proper model that the problem at hand requires. SO, the model should describe the property of interest, the simulation size and simulation time should be chosen accordingly. The hierarchy of the topology file is: the force field, molecules, and last the system level. 
* Information on how we want to **run the simulation** :: **molecular dynamics parameter file (\*mdp)**
 * this file defines the parametrs we need, such as integrator, time steps, number of steps, constraints, etc. (some settings can also depend on the force field, like use the same time steps as used in the force field parametrization.) We need to choose the ensemble we want to use as well, NVE, NVT, NPT, etc. If initial velocities are needed, they can be generated using $E_{kin}=\sum_{i=1}^{N}m_i\frac{v_i^2}{2}=\frac{N_{df}K_BT}{2}$
 
We can use _gmx_ _grompp_ to include everything into a \*.tpr file and then run _gmx mdrun_ to generate our output files.


## Output files
Ensemble of structures, called configurations:
    - *.xtc/*.trr --> trajectory (binary)
    - *.edr --> energy file (binary)
    - *.log --> log file (ASCII)
    - *.gro --> last structure

## Workflow of data production

starting structure --> energy minimization --> solvent and ion relaxation, system equilibration --> data production --> configurations 

For each step we need a seperate *.mdp file.

### GROMACS analysis tools:
- _gmx energy_  to process energy
- _gmx trjconv_ to convert trj files
- _gmx select / gmx make__ndx_ for selection
- _gmx rms_
- _gmx cluster_ for trajectory analysis
- _gmx distance / gmx mindist_ 

### To visualize
- VMD
- Rasmol
- Pymol
- Chimera

## Restraints and Simulations

Types of restraints:
* **Position** restraints
* **Flat-bottomed position** restraints: sphere, cylinder, layer
* **Angle and Dihedral** restraints
* **Distance** restraints: simple harmonic, piecewise linear/harmonic, complex, NMR distance
* **Orientation** restraints

_gmx genrestr_ may be useful in including restraints

### Steps:
1. set up a standard MD simulation
2. add restraints information to the topology file
3. add NMR refinement parameter to mdp file
4. supply a file with restraint coordinates to tpr file (_gmx grompp_)
5. perform the simulation

### Density guided simulations:
1. set up a standard MD simulation
2. have a reference density (.mrc or .ccp4 file) in working_dir and put the name in mdp file
3. add density guided simulation parameter to mdp file
4. perform the simulation